### 練習問題2.71
$n$記号のアルファベットに対するハフマン木があり、  
記号の相対頻度は$1,2,4,...,2^{n−1}$であるとする。  
$n = 5$、$n = 10$の場合の⽊をスケッチせよ。  
そのような木では、(⼀般の$n$について )最も頻度の⾼い記号を符号化するのに何ビット必要になるだろうか。  
最も頻度の低い記号はどうだろうか。

木の回答は保留にする。

In [1]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [2]:
; 符号化処理
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

; 符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [3]:
; ハフマン符号木の生成の記号・頻度ペアのリストから、指定したペアを削除したリストを返す。
(define (remove-set item set)
  (if (null? set) '()
      (if (equal? item (car set)) (remove-set item (cdr set))
          (cons (car set) (remove-set item (cdr set)))
      )
  )
)

; 回答
(define (successive-merge set)
  (define (iter result)
    (cond ((null? result) '())
          ((= (length result) 1) (car result))
          (else
             (let ((top (car result))
                   (next (cadr result))
                   )
               (let (
                     (new-set (remove-set next (remove-set top result)))
                     ;(new-set (cddr result)) ; remove-set手続きを使わなくてもcddrで代用可能
                     )
                   (iter (adjoin-set (make-code-tree top next) new-set))
                 )
               )
           )
        )
    )
  (iter set)
  )

(define (generate-huffman-tree pairs)
    (successive-merge (make-leaf-set pairs)))

In [4]:
; 2^(5-1)
(define pairs1 '((A 16) (B 8) (C 4) (D 2) (E 1)))
(define sample1 (generate-huffman-tree pairs1))
sample1

(((((leaf E 1) (leaf D 2) (E D) 3) (leaf C 4) (E D C) 7) (leaf B 8) (E D C B) 15) (leaf A 16) (E D C B A) 31)

In [5]:
; 単一の記号で動作確認
(display (encode '(A) sample1))
(newline)
(display (encode '(B) sample1))
(newline)
(display (encode '(C) sample1))
(newline)
(display (encode '(D) sample1))
(newline)
(display (encode '(E) sample1))
(newline)

(1)
(0 1)
(0 0 1)
(0 0 0 1)
(0 0 0 0)


In [6]:
; 2^(10-1)
(define pairs2 '((A 512) (B 256) (C 128) (D 64) (E 32) (F 16) (G 8) (H 4) (I 2) (J 1)))
(define sample2 (generate-huffman-tree pairs2))
sample2

((((((((((leaf J 1) (leaf I 2) (J I) 3) (leaf H 4) (J I H) 7) (leaf G 8) (J I H G) 15) (leaf F 16) (J I H G F) 31) (leaf E 32) (J I H G F E) 63) (leaf D 64) (J I H G F E D) 127) (leaf C 128) (J I H G F E D C) 255) (leaf B 256) (J I H G F E D C B) 511) (leaf A 512) (J I H G F E D C B A) 1023)

In [7]:
; 単一の記号で動作確認
(display (encode '(A) sample2))
(newline)
(display (encode '(B) sample2))
(newline)
(display (encode '(C) sample2))
(newline)
(display (encode '(D) sample2))
(newline)
(display (encode '(E) sample2))
(newline)
(display (encode '(F) sample2))
(newline)
(display (encode '(G) sample2))
(newline)
(display (encode '(H) sample2))
(newline)
(display (encode '(I) sample2))
(newline)
(display (encode '(J) sample2))
(newline)

(1)
(0 1)
(0 0 1)
(0 0 0 1)
(0 0 0 0 1)
(0 0 0 0 0 1)
(0 0 0 0 0 0 1)
(0 0 0 0 0 0 0 1)
(0 0 0 0 0 0 0 0 1)
(0 0 0 0 0 0 0 0 0)
